In [5]:
from langchain.vectorstores.redis import Redis

metadata = [
    {
        "user": "john",
        "age": 18,
        "job": "engineer",
        "credit_score": "high",
    },
    {
        "user": "derrick",
        "age": 45,
        "job": "doctor",
        "credit_score": "low",
    },
    {
        "user": "nancy",
        "age": 94,
        "job": "doctor",
        "credit_score": "high",
    },
    {
        "user": "tyler",
        "age": 100,
        "job": "engineer",
        "credit_score": "high",
    },
    {
        "user": "joe",
        "age": 35,
        "job": "dentist",
        "credit_score": "medium",
    },
]

texts = ["foo", "foo", "foo", "bar", "bar"]


Redis.__init__()

Redis.


rds = Redis.from_texts(
    texts,
    embeddings, 
    metadatas=metadata,
    redis_url="redis://localhost:6379",
    index_name="users",
)

rds.index_name

TypeError: __init__() missing 4 required positional arguments: 'self', 'redis_url', 'index_name', and 'embedding'